In [64]:
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import glob
import gc
import math

In [65]:
def cyclical_learning_rate(epoch, lr):
    base_lr = 0.001  # The minimum learning rate
    max_lr = 0.01    # The maximum learning rate
    step_size = 3   # The number of epochs between each cycle

    cycle = math.floor(1 + epoch/(2*step_size))
    x = abs(epoch/step_size - 2*cycle + 1)
    new_lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))

    return new_lr * (6 / (cycle + 5))

# Stanford 40
### Download the data
You can see the zip files if you click the Files tab (looks like a folder symbol on the left of the screen)

In [20]:
!wget http://vision.stanford.edu/Datasets/Stanford40_JPEGImages.zip
!wget http://vision.stanford.edu/Datasets/Stanford40_ImageSplits.zip

--2023-04-14 14:52:21--  http://vision.stanford.edu/Datasets/Stanford40_JPEGImages.zip
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 304771808 (291M) [application/zip]
Saving to: 'Stanford40_JPEGImages.zip'

     0K .......... .......... .......... .......... ..........  0%  158K 31m18s
    50K .......... .......... .......... .......... ..........  0%  315K 23m31s
   100K .......... .......... .......... .......... ..........  0%  316K 20m54s
   150K .......... .......... .......... .......... ..........  0%  321K 19m32s
   200K .......... .......... .......... .......... ..........  0% 11.1M 15m43s
   250K .......... .......... .......... .......... ..........  0%  326K 15m37s
   300K .......... .......... .......... .......... ..........  0% 11.3M 13m27s
   350K .......... .......... .......... .......... ..........  0% 1

### Unzip it

In [27]:
os.system("unzip Stanford40_JPEGImages.zip -d Stanford40/")
os.system("unzip Stanford40_ImageSplits.zip -d Stanford40/")

1

## Read the train and test splits, combine them and make better splits to help training networks easier.

In [66]:
keep_stanford40 = ["applauding", "climbing", "drinking", "jumping", "pouring_liquid", "riding_a_bike", "riding_a_horse", 
        "running", "shooting_an_arrow", "smoking", "throwing_frisby", "waving_hands"]
with open('Stanford40/ImageSplits/train.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    sf_train_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    sf_train_labels = ['_'.join(name.split('_')[:-1]) for name in sf_train_files]

with open('Stanford40/ImageSplits/test.txt', 'r') as f:
    # We won't use these splits but split them ourselves
    sf_test_files = [file_name for file_name in list(map(str.strip, f.readlines())) if '_'.join(file_name.split('_')[:-1]) in keep_stanford40]
    sf_test_labels = ['_'.join(name.split('_')[:-1]) for name in sf_test_files]

# Combine the splits and split for keeping more images in the training set than the test set.
sf_all_files = sf_train_files + sf_test_files
sf_all_labels = sf_train_labels + sf_test_labels
sf_train_validation_files, sf_test_files = train_test_split(sf_all_files, test_size=0.1, random_state=0, stratify=sf_all_labels)
sf_train_validation_labels = ['_'.join(name.split('_')[:-1]) for name in sf_train_validation_files]
sf_train_files, sf_validation_files = train_test_split(sf_train_validation_files, test_size=0.1, random_state=0, stratify=sf_train_validation_labels)

sf_train_labels = ['_'.join(name.split('_')[:-1]) for name in sf_train_files]
sf_train_labels = list(map(lambda x: keep_stanford40.index(x), sf_train_labels))
sf_test_labels = ['_'.join(name.split('_')[:-1]) for name in sf_test_files]
sf_test_labels = list(map(lambda x: keep_stanford40.index(x), sf_test_labels))
sf_validation_labels = ['_'.join(name.split('_')[:-1]) for name in sf_validation_files]
sf_validation_labels = list(map(lambda x: keep_stanford40.index(x), sf_validation_labels))

print(f'Train files ({len(sf_train_files)}):\n\t{sf_train_files}')
print(f'Train labels ({len(sf_train_labels)}):\n\t{map(str, sf_train_labels)}\n'\
      f'Train Distribution:{list(Counter(sorted(sf_train_labels)).items())}\n')
print(f'Test files ({len(sf_test_files)}):\n\t{sf_test_files}')
print(f'Test labels ({len(sf_test_labels)}):\n\t{map(str, sf_test_labels)}\n'\
      f'Test Distribution:{list(Counter(sorted(sf_test_labels)).items())}\n')
print(f'Validation files ({len(sf_validation_files)}):\n\t{sf_validation_files}')
print(f'Validation labels ({len(sf_validation_labels)}):\n\t{map(str, sf_validation_labels)}\n'\
      f'Validation Distribution:{list(Counter(sorted(sf_validation_labels)).items())}\n')

sf_action_categories = sorted(list(set(sf_train_labels)))
print(f'Action categories ({len(sf_action_categories)}):\n{sf_action_categories}')

Train files (2459):
	['riding_a_bike_140.jpg', 'running_196.jpg', 'jumping_005.jpg', 'riding_a_bike_282.jpg', 'climbing_130.jpg', 'running_168.jpg', 'waving_hands_168.jpg', 'riding_a_horse_161.jpg', 'climbing_048.jpg', 'pouring_liquid_053.jpg', 'riding_a_bike_292.jpg', 'running_039.jpg', 'drinking_220.jpg', 'drinking_005.jpg', 'smoking_030.jpg', 'riding_a_bike_286.jpg', 'drinking_025.jpg', 'applauding_215.jpg', 'smoking_117.jpg', 'waving_hands_196.jpg', 'applauding_140.jpg', 'riding_a_horse_292.jpg', 'throwing_frisby_013.jpg', 'smoking_172.jpg', 'applauding_005.jpg', 'climbing_210.jpg', 'pouring_liquid_104.jpg', 'riding_a_bike_128.jpg', 'waving_hands_041.jpg', 'riding_a_bike_083.jpg', 'shooting_an_arrow_048.jpg', 'drinking_034.jpg', 'drinking_223.jpg', 'drinking_100.jpg', 'running_188.jpg', 'pouring_liquid_031.jpg', 'smoking_184.jpg', 'jumping_225.jpg', 'jumping_272.jpg', 'riding_a_horse_069.jpg', 'jumping_273.jpg', 'smoking_178.jpg', 'climbing_253.jpg', 'jumping_185.jpg', 'pouring_liq

Resize images

In [67]:
sf_train_images = []
for img_nr in range(len(sf_train_files)):
  img = cv2.imread(f'Stanford40/JPEGImages/{sf_train_files[img_nr]}')
  img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
  cv2.imwrite(f'Stanford40/JPEGImages/{sf_train_files[img_nr]}', img)
  sf_train_images.append(img)
sf_train_images = np.array(sf_train_images) / 255.0
sf_train_labels = np.array(sf_train_labels)

sf_test_images = []
for img_nr in range(len(sf_test_files)):
  img = cv2.imread(f'Stanford40/JPEGImages/{sf_test_files[img_nr]}')
  img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
  cv2.imwrite(f'Stanford40/JPEGImages/{sf_test_files[img_nr]}', img)
  sf_test_images.append(img)
sf_test_images = np.array(sf_test_images) / 255.0
sf_test_labels = np.array(sf_test_labels)

sf_validation_images = []
for img_nr in range(len(sf_validation_files)):
  img = cv2.imread(f'Stanford40/JPEGImages/{sf_validation_files[img_nr]}')
  img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
  cv2.imwrite(f'Stanford40/JPEGImages/{sf_validation_files[img_nr]}', img)
  sf_validation_images.append(img)
sf_validation_images = np.array(sf_validation_images) / 255.0
sf_validation_labels = np.array(sf_validation_labels)

sf_train_validation_files = np.concatenate((sf_train_images, sf_validation_images))
sf_train_validation_labels = np.concatenate((sf_train_labels, sf_validation_labels))



### Visualize a photo from the training files and also print its label

In [30]:
image_no = 232  # change this to a number between [0, 1200] and you can see a different training image
img = cv2.imread(f'Stanford40/JPEGImages/{sf_train_files[image_no]}')
print(f'An image with the label - {sf_train_labels[image_no]}')
#cv2_imshow(img)

An image with the label - 3


Make Stanford CNN

In [31]:
#poging 1
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Base_Model = tf.keras.Sequential([  
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='same', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, (3, 3), 2, padding='valid', activation='relu'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

opti = tf.keras.optimizers.Adam(learning_rate=0.001)

Base_Model.compile(optimizer=opti,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Base_Model.build(batch_input_shape)
Base_Model.summary()

history_base_model = Base_Model.fit(sf_train_images, sf_train_labels, epochs=25, validation_data=(sf_validation_images, sf_validation_labels))

validate_loss, validate_acc = Base_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

print(validate_loss)
print(validate_acc)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (32, 112, 112, 32)        2432      
                                                                 
 re_lu (ReLU)                (32, 112, 112, 32)        0         
                                                                 
 batch_normalization (BatchN  (32, 112, 112, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (32, 54, 54, 32)          25632     
                                                                 
 re_lu_1 (ReLU)              (32, 54, 54, 32)          0         
                                                                 
 batch_normalization_1 (Batc  (32, 54, 54, 32)         128       
 hNormalization)                                        

C:\Users\jeroe\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


77/77 [==============================] - 28s 341ms/step - loss: 2.5066 - accuracy: 0.1118 - val_loss: 2.4727 - val_accuracy: 0.0949
Epoch 2/25
77/77 [==============================] - 25s 324ms/step - loss: 2.3852 - accuracy: 0.1704 - val_loss: 2.5350 - val_accuracy: 0.1022
Epoch 3/25
77/77 [==============================] - 25s 320ms/step - loss: 2.2714 - accuracy: 0.2123 - val_loss: 2.5663 - val_accuracy: 0.1350
Epoch 4/25
77/77 [==============================] - 24s 315ms/step - loss: 2.1343 - accuracy: 0.2611 - val_loss: 2.3922 - val_accuracy: 0.1715
Epoch 5/25
77/77 [==============================] - 24s 314ms/step - loss: 2.0407 - accuracy: 0.3017 - val_loss: 2.2625 - val_accuracy: 0.2153
Epoch 6/25
77/77 [==============================] - 24s 308ms/step - loss: 1.9745 - accuracy: 0.3188 - val_loss: 2.4801 - val_accuracy: 0.1679
Epoch 7/25
77/77 [==============================] - 24s 310ms/step - loss: 1.8451 - accuracy: 0.3778 - val_loss: 2.4011 - val_accuracy: 0.1971
Epoch 8/25

In [32]:
#poging 2
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Base_Model = tf.keras.Sequential([  
    tf.keras.layers.Conv2D(filter_count, kernel_size, 2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, 3, padding='valid', activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l=0.05)),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    #tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

opti = tf.keras.optimizers.Adam(learning_rate=0.0001)

Base_Model.compile(optimizer=opti,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Base_Model.build(batch_input_shape)
Base_Model.summary()

history_base_model = Base_Model.fit(sf_train_images, sf_train_labels, epochs=15, validation_data=(sf_validation_images, sf_validation_labels))

validate_loss, validate_acc = Base_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

print(validate_loss)
print(validate_acc)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (32, 112, 112, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2D  (32, 56, 56, 32)         0         
 )                                                               
                                                                 
 batch_normalization_6 (Batc  (32, 56, 56, 32)         128       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (32, 18, 18, 32)          25632     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 6, 6, 32)           0         
 2D)                                                             
                                                      

In [33]:
#poging 3
filter_count = 32
kernel_size = (3, 3)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Base_Model = tf.keras.Sequential([  
    tf.keras.layers.Conv2D(filter_count, kernel_size, 2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, (5, 5), 3, padding='valid', activation='relu', kernel_regularizer=tf.keras.regularizers.L1(l=0.05)),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(filter_count, (3, 3), strides, padding='valid', activation='relu'),
    #tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.BatchNormalization(),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(12, activation ='softmax')
])

opti = tf.keras.optimizers.Adam(learning_rate=0.001)

Base_Model.compile(optimizer=opti,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Base_Model.build(batch_input_shape)
Base_Model.summary()

history_base_model = Base_Model.fit(sf_train_images, sf_train_labels, epochs=15, validation_data=(sf_validation_images, sf_validation_labels))

validate_loss, validate_acc = Base_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

print(validate_loss)
print(validate_acc)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (32, 112, 112, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (32, 56, 56, 32)         0         
 2D)                                                             
                                                                 
 batch_normalization_9 (Batc  (32, 56, 56, 32)         128       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (32, 18, 18, 32)          25632     
                                                                 
 max_pooling2d_3 (MaxPooling  (32, 6, 6, 32)           0         
 2D)                                                             
                                                      

In [34]:
#poging 4
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Base_Model = tf.keras.Sequential([  
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, 3, padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),
    
    #tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='valid', activation='relu'),
    #tf.keras.layers.BatchNormalization(),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

Base_Model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Base_Model.build(batch_input_shape)
Base_Model.summary()

history_base_model = Base_Model.fit(sf_train_images, sf_train_labels, epochs=15, validation_data=(sf_validation_images, sf_validation_labels))

validate_loss, validate_acc = Base_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

Base_Model.save_weights('Weights/StanfordModel')

print(validate_loss)

print(validate_acc)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (32, 112, 112, 32)        2432      
                                                                 
 max_pooling2d_4 (MaxPooling  (32, 37, 37, 32)         0         
 2D)                                                             
                                                                 
 batch_normalization_12 (Bat  (32, 37, 37, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (32, 11, 11, 32)          25632     
                                                                 
 max_pooling2d_5 (MaxPooling  (32, 3, 3, 32)           0         
 2D)                                                             
                                                      

In [68]:
#poging 5
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Best_Stanford_Model = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(0.05),
    tf.keras.layers.RandomFlip("horizontal"),
    #tf.keras.layers.RandomBrightness(0.01),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, 3, padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

Best_Stanford_Model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Best_Stanford_Model.build(batch_input_shape)
Best_Stanford_Model.summary()

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(cyclical_learning_rate)

history_best_stanford_model = Best_Stanford_Model.fit(sf_train_images, sf_train_labels, epochs=25, validation_data=(sf_validation_images, sf_validation_labels))

validate_loss, validate_acc = Best_Stanford_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

Best_Stanford_Model.save_weights('Weights/StanfordModel')

print(validate_loss)
print(validate_acc)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast_16 (RandomC  (32, 224, 224, 3)        0         
 ontrast)                                                        
                                                                 
 random_flip_16 (RandomFlip)  (32, 224, 224, 3)        0         
                                                                 
 random_rotation_16 (RandomR  (32, 224, 224, 3)        0         
 otation)                                                        
                                                                 
 conv2d_34 (Conv2D)          (32, 112, 112, 32)        2432      
                                                                 
 max_pooling2d_34 (MaxPoolin  (32, 37, 37, 32)         0         
 g2D)                                                            
                                                     

C:\Users\Gideon\venv\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


77/77 [==============================] - 14s 174ms/step - loss: 2.4984 - accuracy: 0.1472 - val_loss: 2.5096 - val_accuracy: 0.1277
Epoch 2/25
77/77 [==============================] - 13s 165ms/step - loss: 2.2475 - accuracy: 0.2330 - val_loss: 2.8620 - val_accuracy: 0.1058
Epoch 3/25
77/77 [==============================] - 12s 160ms/step - loss: 2.1703 - accuracy: 0.2672 - val_loss: 2.7938 - val_accuracy: 0.1168
Epoch 4/25
77/77 [==============================] - 12s 159ms/step - loss: 2.0883 - accuracy: 0.2859 - val_loss: 2.8869 - val_accuracy: 0.1168
Epoch 5/25
77/77 [==============================] - 12s 161ms/step - loss: 2.0399 - accuracy: 0.3058 - val_loss: 2.4983 - val_accuracy: 0.1898
Epoch 6/25
77/77 [==============================] - 12s 161ms/step - loss: 2.0048 - accuracy: 0.3131 - val_loss: 2.2579 - val_accuracy: 0.2409
Epoch 7/25
77/77 [==============================] - 12s 160ms/step - loss: 1.9261 - accuracy: 0.3432 - val_loss: 2.1626 - val_accuracy: 0.2920
Epoch 8/25

In [71]:
#poging 5 with cyclical learning
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Base_Model = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(0.05),
    tf.keras.layers.RandomFlip("horizontal"),
    #tf.keras.layers.RandomBrightness(0.01),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, 3, padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

Base_Model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Base_Model.build(batch_input_shape)
Base_Model.summary()

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(cyclical_learning_rate)

history_base_model = Base_Model.fit(sf_train_images, sf_train_labels, epochs=25, validation_data=(sf_validation_images, sf_validation_labels), callbacks=[lr_scheduler])

validate_loss, validate_acc = Base_Model.evaluate(sf_validation_images,  sf_validation_labels, verbose=2)

Base_Model.save_weights('Weights/StanfordModel')

print(validate_loss)
print(validate_acc)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast_19 (RandomC  (32, 224, 224, 3)        0         
 ontrast)                                                        
                                                                 
 random_flip_19 (RandomFlip)  (32, 224, 224, 3)        0         
                                                                 
 random_rotation_19 (RandomR  (32, 224, 224, 3)        0         
 otation)                                                        
                                                                 
 conv2d_40 (Conv2D)          (32, 112, 112, 32)        2432      
                                                                 
 max_pooling2d_40 (MaxPoolin  (32, 37, 37, 32)         0         
 g2D)                                                            
                                                     

In [37]:
try: del sf_train_images
except: print("sf_train_images is not defined")
try: del sf_train_labels
except: print("sf_train_labels is not defined")
try: del sf_validation_images
except: print("sf_validation_images is not defined")
try: del sf_validation_labels
except: print("sf_validation_labels is not defined")
try: del Base_Model
except: print("Base_Model is not defined")

gc.collect()

20164

# Human Motion Database 51 (HMDB51)
### Download the dataset

In [38]:
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2023-04-14 15:28:29--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2023-04-14 15:28:29--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: 'hmdb51_org.rar'

     0K .......... .......... .......... .......... ..........  0%  261K 2h12m
    50K .......... .......... .......... .......... ..........  0%  527K 98m58s
   100K .......... .......... .......... .......... ..........  0% 11.7M 66m56s
   150K .......... .......... .

# Extract and organize the data.

In [7]:
os.system("mkdir video_data test_train_splits")
os.system("unrar e test_train_splits.rar test_train_splits")
os.system("del test_train_splits.rar")
os.system("unrar e hmdb51_org.rar")
os.system("del hmdb51_org.rar")
os.system("move *.rar video_data")
keep_hmdb51 = ["clap", "climb", "drink", "jump", "pour", "ride_bike", "ride_horse", 
        "run", "shoot_bow", "smoke", "throw", "wave"]
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    if foldername in keep_hmdb51:
      # extract only the relevant classes for the assignment.
      os.system("mkdir video_data\\" + foldername)
      os.system("unrar e video_data\\"+ files + " video_data\\"+foldername)

os.system("del video_data\*.rar")


0

# Split the dataset into train and test 

In [60]:
TRAIN_TAG, TEST_TAG = 1, 2
hm_train_files, hm_test_files = [], []
hm_train_labels, hm_test_labels = [], []
split_pattern_name = f"*test_split1.txt"
split_pattern_path = os.path.join('test_train_splits', split_pattern_name)
annotation_paths = glob.glob(split_pattern_path)
for filepath in annotation_paths:
    class_name = '_'.join(filepath.split('\\')[-1].split('_')[:-2])
    if class_name not in keep_hmdb51:
        continue  # skipping the classes that we won't use.
    with open(filepath) as fid:
        lines = fid.readlines()
    for line in lines:
        video_filename, tag_string = line.split()
        tag = int(tag_string)
        if tag == TRAIN_TAG:
            hm_train_files.append(video_filename)
            hm_train_labels.append(class_name)
        elif tag == TEST_TAG:
            hm_test_files.append(video_filename)
            hm_test_labels.append(class_name)

hm_train_files, hm_val_files, hm_train_labels, hm_val_labels = train_test_split(hm_train_files, hm_train_labels, test_size=0.1)

print(f'Train files ({len(hm_train_files)}):\n\t{hm_train_files}')
print(f'Train labels ({len(hm_train_labels)}):\n\t{hm_train_labels}\n'\
      f'Train Distribution:{list(Counter(sorted(hm_train_labels)).items())}\n')
print(f'Test files ({len(hm_test_files)}):\n\t{hm_test_files}')
print(f'Test labels ({len(hm_test_labels)}):\n\t{hm_test_labels}\n'\
      f'Test Distribution:{list(Counter(sorted(hm_test_labels)).items())}\n')
print(f'Validation files ({len(hm_val_files)}):\n\t{hm_val_files}')
print(f'Validation labels ({len(hm_val_labels)}):\n\t{hm_val_labels}\n'\
      f'Validation Distribution:{list(Counter(sorted(hm_val_labels)).items())}\n')
action_categories = sorted(list(set(hm_train_labels)))
print(f'Action categories ({len(action_categories)}):\n{action_categories}')

hm_train_labels_nr = np.array(list(map(lambda x: action_categories.index(x), hm_train_labels)))
hm_test_labels_nr = np.array(list(map(lambda x: action_categories.index(x), hm_test_labels)))
hm_val_labels_nr = np.array(list(map(lambda x: action_categories.index(x), hm_val_labels)))

Train files (756):
	['Kraftwerk_-_Tour_de_france_1983_Alternative_video_ride_bike_f_cm_np4_fr_med_0.avi', 'Budam_-_Clap_Hands_clap_u_nm_np1_fr_med_2.avi', 'The_Fugitive_2_run_f_cm_np1_fr_med_1.avi', 'Bier_richtig_einschenken_pour_u_cm_np1_fr_med_1.avi', 'CSC_and_the_2007__Sydney_to_the_Gong__bike_ride_ride_bike_f_cm_np1_ri_med_1.avi', 'Gaiwan_tea_ceremony_pour_u_cm_np1_fr_med_2.avi', 'Tour_de_France_2003_-_Armstrong_attacks_Ullrich_after_Fall_ride_bike_f_cm_np1_ba_med_3.avi', 'Erikafrontrecurve6june08_shoot_bow_u_nm_np1_fr_med_0.avi', 'Kraftwerk_-_Tour_de_france_1983_Alternative_video_ride_bike_f_cm_np1_le_med_6.avi', 'amazingballthrowingtricks_throw_f_cm_np1_ri_med_2.avi', 'Two_Towers_1_run_u_cm_np2_fr_bad_6.avi', 'OSSER_-_Qualboro_light_-_Marlboro_Verarschung_smoke_h_cm_np1_le_bad_0.avi', '50_FIRST_DATES_wave_u_cm_np1_fr_goo_30.avi', 'crazy_german_guy_shows_how_do_they_pour_a_wheat_beer_pour_u_cm_np1_fr_goo_1.avi', 'RETURN_OF_THE_KING_run_u_cm_np1_fr_bad_46.avi', 'Veoh_Alpha_Dog_1_sm

In [61]:
hm_train_images = []
hm_train_flow = []

hm_test_images = []
hm_test_flow = []

hm_val_images = []
hm_val_flow = []

cur_percentage = 0.99

def calcFlow(path, fname):
  cap = cv2.VideoCapture(path)

  frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

  first_frame = frame_count - 2

  cap.set(cv2.CAP_PROP_POS_FRAMES, first_frame-1)
  ret, frame1 = cap.read()


  cap.set(cv2.CAP_PROP_POS_FRAMES, first_frame)
  ret, frame2 = cap.read()

  prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
  next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
  hsv = np.zeros_like(frame1)
  hsv[..., 0] = ang*180/np.pi/2
  hsv[..., 1] = 255
  hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
  bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
  frame2 = cv2.resize(frame2, (224, 224), interpolation = cv2.INTER_AREA)
  bgr = cv2.resize(bgr, (224, 224), interpolation = cv2.INTER_AREA)
    
  cv2.imwrite('video_data\\images\\' + fname[:-3] + '.png', frame2)
  cv2.imwrite('video_data\\flow\\' + fname[:-3] + '_flow.png', bgr)

  return (frame2, bgr)

for i, train_file in enumerate(hm_train_files):
  path = 'video_data\\' + hm_train_labels[i] + '\\' + train_file
  img, flow = calcFlow(path, train_file)
  hm_train_images.append(img)
  hm_train_flow.append(flow)

hm_train_images = np.array(hm_train_images) / 255.0
hm_train_flow = np.array(hm_train_flow) / 255.0

print("done train files")

for j, test_file in enumerate(hm_test_files):
  path = 'video_data\\' + hm_test_labels[j] + '\\' + test_file
  img, flow = calcFlow(path, test_file)
  hm_test_images.append(img)
  hm_test_flow.append(flow)

hm_test_images = np.array(hm_test_images) / 255.0
hm_test_flow = np.array(hm_test_flow) / 255.0

print("done test files")

for k, val_file in enumerate(hm_val_files):
  path = 'video_data\\' + hm_val_labels[k] + '\\' + val_file
  img, flow = calcFlow(path, val_file)
  hm_val_images.append(img)
  hm_val_flow.append(flow)

hm_val_images = np.array(hm_val_images) / 255.0
hm_val_flow = np.array(hm_val_flow) / 255.0

print("done val files")


done train files
done test files
done val files


# Pretrain Model

In [69]:
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Pre_Trained_Base_Model = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(0.05),
    tf.keras.layers.RandomFlip("horizontal"),
    #tf.keras.layers.RandomBrightness(0.01),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(filter_count, kernel_size, 3, padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D((3, 3)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

Pre_Trained_Base_Model.load_weights('Weights/StanfordModel')

opti = tf.keras.optimizers.Adam(learning_rate=0.00001)

Pre_Trained_Base_Model.compile(optimizer=opti,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Pre_Trained_Base_Model.build(batch_input_shape)
Pre_Trained_Base_Model.summary()

history_pre_trained_model = Pre_Trained_Base_Model.fit(np.array(hm_train_images), np.array(hm_train_labels_nr), epochs=15, validation_data=(np.array(hm_val_images), np.array(hm_val_labels_nr)))

validate_loss, validate_acc = Pre_Trained_Base_Model.evaluate(np.array(hm_val_images),  np.array(hm_val_labels_nr), verbose=2)

Pre_Trained_Base_Model.save_weights('Weights/PretrainedModel')

print(validate_loss)
print(validate_acc)


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast_17 (RandomC  (32, 224, 224, 3)        0         
 ontrast)                                                        
                                                                 
 random_flip_17 (RandomFlip)  (32, 224, 224, 3)        0         
                                                                 
 random_rotation_17 (RandomR  (32, 224, 224, 3)        0         
 otation)                                                        
                                                                 
 conv2d_36 (Conv2D)          (32, 112, 112, 32)        2432      
                                                                 
 max_pooling2d_36 (MaxPoolin  (32, 37, 37, 32)         0         
 g2D)                                                            
                                                     

# Optical Flow

In [70]:
filter_count = 32
kernel_size = (5, 5)
strides = (2)
batch_input_shape = (32, 224, 224, 3)

Optical_Flow_Model = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(0.05),
    tf.keras.layers.RandomFlip("horizontal"),
    #tf.keras.layers.RandomBrightness(0.01),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Conv2D(filter_count, kernel_size, strides, activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(filter_count, (3, 3), 1, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(12, activation ='softmax')
])

opti = tf.keras.optimizers.Adam(learning_rate=0.001)

Optical_Flow_Model.compile(optimizer=opti,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Optical_Flow_Model.build(batch_input_shape)
Optical_Flow_Model.summary()

history_optical_flow_model = Optical_Flow_Model.fit(np.array(hm_train_flow), np.array(hm_train_labels_nr), epochs=15, validation_data=(np.array(hm_val_flow), np.array(hm_val_labels_nr)))

validate_loss, validate_acc = Optical_Flow_Model.evaluate(np.array(hm_val_flow),  np.array(hm_val_labels_nr), verbose=2)

Optical_Flow_Model.save_weights('Weights/OpticalFlowModel')

print(validate_loss)
print(validate_acc)


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_contrast_18 (RandomC  (32, 224, 224, 3)        0         
 ontrast)                                                        
                                                                 
 random_flip_18 (RandomFlip)  (32, 224, 224, 3)        0         
                                                                 
 random_brightness_1 (Random  (32, 224, 224, 3)        0         
 Brightness)                                                     
                                                                 
 random_rotation_18 (RandomR  (32, 224, 224, 3)        0         
 otation)                                                        
                                                                 
 conv2d_38 (Conv2D)          (32, 110, 110, 32)        2432      
                                                     

In [53]:
# Twostream
Pre_Trained_Base_Model.trainable = False
Optical_Flow_Model.trainable = False

fusionLayers = tf.keras.layers.Add()([
    Pre_Trained_Base_Model.output,
    Optical_Flow_Model.output
])

# for bonus connect with 1x1 convolutionial layer change this to conv layer
fusionLayers = tf.keras.layers.Dense(12, activation='softmax')(fusionLayers)

print(Pre_Trained_Base_Model.input_shape)
print(Optical_Flow_Model.input_shape)

fusionModel = tf.keras.models.Model([
    Pre_Trained_Base_Model.input,
    Optical_Flow_Model.input
], fusionLayers)

opti = tf.keras.optimizers.Adam(learning_rate=0.001)

fusionModel.compile(optimizer=opti, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

fusionHistory = fusionModel.fit([hm_train_images,
                                 hm_train_flow],
                                hm_train_labels_nr,
                                validation_data=([hm_val_images, hm_val_flow],
                                                 hm_val_labels_nr),
                                epochs=20)

fusionModel.save_weights('Weights/FusionModel')

Pre_Trained_Base_Model.trainable = True
Optical_Flow_Model.trainable = True

(None, 224, 224, 3)
(None, 224, 224, 3)
Epoch 1/20
24/24 [==============================] - 2s 72ms/step - loss: 2.4858 - accuracy: 0.0635 - val_loss: 2.4775 - val_accuracy: 0.1071
Epoch 2/20
24/24 [==============================] - 2s 66ms/step - loss: 2.4739 - accuracy: 0.0939 - val_loss: 2.4707 - val_accuracy: 0.1190
Epoch 3/20
24/24 [==============================] - 2s 67ms/step - loss: 2.4673 - accuracy: 0.0926 - val_loss: 2.4626 - val_accuracy: 0.1071
Epoch 4/20
24/24 [==============================] - 2s 67ms/step - loss: 2.4586 - accuracy: 0.1019 - val_loss: 2.4564 - val_accuracy: 0.1548
Epoch 5/20
24/24 [==============================] - 2s 67ms/step - loss: 2.4523 - accuracy: 0.1058 - val_loss: 2.4498 - val_accuracy: 0.1905
Epoch 6/20
24/24 [==============================] - 2s 67ms/step - loss: 2.4452 - accuracy: 0.1243 - val_loss: 2.4428 - val_accuracy: 0.2143
Epoch 7/20
24/24 [==============================] - 2s 66ms/step - loss: 2.4373 - accuracy: 0.1310 - val_loss: 2.4

In [72]:
# Stanford Test
Best_Stanford_Result = Best_Stanford_Model.fit(np.concatenate((hm_train_images, hm_val_images)), np.concatenate((hm_train_labels_nr, hm_val_labels_nr)), epochs=15)

test_loss, test_acc = Best_Stanford_Model.evaluate(hm_test_images,  hm_test_labels_nr, verbose=2)

print(test_loss)
print(test_acc)

Best_Stanford_Model.save_weights('Weights/BestStandfordModel')

print(history_best_stanford_model.history)

Epoch 1/15
27/27 [==============================] - 3s 112ms/step - loss: 2.4467 - accuracy: 0.2036
Epoch 2/15
27/27 [==============================] - 3s 111ms/step - loss: 2.0726 - accuracy: 0.3179
Epoch 3/15
27/27 [==============================] - 3s 113ms/step - loss: 1.9660 - accuracy: 0.3286
Epoch 4/15
27/27 [==============================] - 3s 113ms/step - loss: 1.8519 - accuracy: 0.3643
Epoch 5/15
27/27 [==============================] - 3s 111ms/step - loss: 1.7941 - accuracy: 0.4060
Epoch 6/15
27/27 [==============================] - 3s 107ms/step - loss: 1.6595 - accuracy: 0.4667
Epoch 7/15
27/27 [==============================] - 3s 104ms/step - loss: 1.6188 - accuracy: 0.4655
Epoch 8/15
27/27 [==============================] - 3s 103ms/step - loss: 1.6525 - accuracy: 0.4369
Epoch 9/15
27/27 [==============================] - 3s 103ms/step - loss: 1.4965 - accuracy: 0.5083
Epoch 10/15
27/27 [==============================] - 3s 103ms/step - loss: 1.4839 - accuracy: 0.5036

In [54]:
# Pretrained Test
Pretrained_Result = Pre_Trained_Base_Model.fit(np.concatenate((hm_train_images, hm_val_images)), np.concatenate((hm_train_labels_nr, hm_val_labels_nr)), epochs=15)

test_loss, test_acc = Pre_Trained_Base_Model.evaluate(hm_test_images,  hm_test_labels_nr, verbose=2)

print(test_loss)
print(test_acc)

Pre_Trained_Base_Model.save_weights('Weights/BestPretrainedModel')

print(history_pre_trained_model.history)

Epoch 1/15
27/27 [==============================] - 2s 85ms/step - loss: 1.7787 - accuracy: 0.4024
Epoch 2/15
27/27 [==============================] - 2s 86ms/step - loss: 1.7787 - accuracy: 0.3976
Epoch 3/15
27/27 [==============================] - 2s 87ms/step - loss: 1.7599 - accuracy: 0.4012
Epoch 4/15
27/27 [==============================] - 2s 85ms/step - loss: 1.7247 - accuracy: 0.4440
Epoch 5/15
27/27 [==============================] - 2s 83ms/step - loss: 1.7238 - accuracy: 0.4238
Epoch 6/15
27/27 [==============================] - 2s 83ms/step - loss: 1.6850 - accuracy: 0.4560
Epoch 7/15
27/27 [==============================] - 2s 83ms/step - loss: 1.6735 - accuracy: 0.4345
Epoch 8/15
27/27 [==============================] - 2s 83ms/step - loss: 1.6430 - accuracy: 0.4464
Epoch 9/15
27/27 [==============================] - 2s 83ms/step - loss: 1.6167 - accuracy: 0.4786
Epoch 10/15
27/27 [==============================] - 2s 83ms/step - loss: 1.5662 - accuracy: 0.4714
Epoch 11/

In [55]:
# Optical Flow Test
Optical_Flow_Result = Optical_Flow_Model.fit(np.concatenate((hm_train_flow, hm_val_flow)), np.concatenate((hm_train_labels_nr, hm_val_labels_nr)), epochs=15)

test_loss, test_acc = Optical_Flow_Model.evaluate(hm_test_flow,  hm_test_labels_nr, verbose=2)

print(test_loss)
print(test_acc)

Optical_Flow_Model.save_weights('Weights/BestFlowModel')

print(history_optical_flow_model.history)

Epoch 1/15
27/27 [==============================] - 4s 129ms/step - loss: 1.9395 - accuracy: 0.3619
Epoch 2/15
27/27 [==============================] - 3s 129ms/step - loss: 1.8649 - accuracy: 0.3738
Epoch 3/15
27/27 [==============================] - 4s 130ms/step - loss: 1.8522 - accuracy: 0.3750
Epoch 4/15
27/27 [==============================] - 4s 133ms/step - loss: 1.8037 - accuracy: 0.4060
Epoch 5/15
27/27 [==============================] - 4s 132ms/step - loss: 1.7581 - accuracy: 0.4119
Epoch 6/15
27/27 [==============================] - 4s 133ms/step - loss: 1.7465 - accuracy: 0.4357
Epoch 7/15
27/27 [==============================] - 4s 134ms/step - loss: 1.6865 - accuracy: 0.4560
Epoch 8/15
27/27 [==============================] - 4s 135ms/step - loss: 1.6239 - accuracy: 0.4631
Epoch 9/15
27/27 [==============================] - 4s 133ms/step - loss: 1.6350 - accuracy: 0.4667
Epoch 10/15
27/27 [==============================] - 4s 132ms/step - loss: 1.5726 - accuracy: 0.4881

In [56]:
# Two Stream Test
Pre_Trained_Base_Model.trainable = False
Optical_Flow_Model.trainable = False

Fusion_Result = fusionModel.fit([np.concatenate((hm_train_images,hm_val_images)), np.concatenate((hm_train_flow,hm_val_flow))], np.concatenate((hm_train_labels_nr, hm_val_labels_nr)), epochs=20)

test_loss, test_acc = fusionModel.evaluate([hm_test_images, hm_test_flow],  hm_test_labels_nr, verbose=2)

print(test_loss)
print(test_acc)

fusionModel.save_weights('Weights/BestFusionModel')

print(fusionHistory.history)

Epoch 1/20
27/27 [==============================] - 2s 71ms/step - loss: 2.1533 - accuracy: 0.4167
Epoch 2/20
27/27 [==============================] - 2s 65ms/step - loss: 2.1082 - accuracy: 0.5155
Epoch 3/20
27/27 [==============================] - 2s 65ms/step - loss: 2.0715 - accuracy: 0.5464
Epoch 4/20
27/27 [==============================] - 2s 65ms/step - loss: 2.0444 - accuracy: 0.5702
Epoch 5/20
27/27 [==============================] - 2s 63ms/step - loss: 2.0276 - accuracy: 0.6107
Epoch 6/20
27/27 [==============================] - 2s 64ms/step - loss: 1.9906 - accuracy: 0.6000
Epoch 7/20
27/27 [==============================] - 2s 66ms/step - loss: 1.9614 - accuracy: 0.6286
Epoch 8/20
27/27 [==============================] - 2s 67ms/step - loss: 1.9458 - accuracy: 0.6167
Epoch 9/20
27/27 [==============================] - 2s 62ms/step - loss: 1.9066 - accuracy: 0.6286
Epoch 10/20
27/27 [==============================] - 2s 61ms/step - loss: 1.9041 - accuracy: 0.6429
Epoch 11/